## Part 1: Database and Jupyter Notebook Set Up
### Terminal Command to Import Data
```bash
mongoimport --db uk_food --collection establishments --file resources/establishments.json --jsonArray
```


In [ ]:
# Import necessary libraries
from pymongo import MongoClient
import pprint

In [ ]:
# Create a MongoClient instance
client = MongoClient()

In [ ]:
# List all databases to confirm uk_food is listed
print(client.list_database_names())

In [ ]:
# Access the uk_food database and list its collections
db = client['uk_food']
print(db.list_collection_names())

In [ ]:
# Access and preview a document from the establishments collection
establishments = db['establishments']
pprint.pprint(establishments.find_one())

In [ ]:
# Assign the establishments collection to a variable for use
establishments = db['establishments']

# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database


In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query =

# Use count_documents to display the number of documents in the result

# Display the first document in the results using pprint


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London in the LocalAuthorityName and has a RatingValue greater than or equal to 4.
query =

# Use count_documents to display the number of documents in the result

# Display the first document in the results using pprint


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude =
longitude =

query =
sort =
limit =

# Print the results


In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


## Part 2: Data Cleanup and Modification
### Step 1: Add a new halal restaurant (Penang Flavours) to the database

In [ ]:
# New restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}
establishments.insert_one(new_restaurant)

### Step 2: Find BusinessTypeID for 'Restaurant/Cafe/Canteen'

In [ ]:
pprint.pprint(establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessType": 1, "BusinessTypeID": 1, "_id": 0}
))

### Step 3: Update the new restaurant with correct BusinessTypeID

In [ ]:
# Replace with actual ID from above after running
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": "1"}}  # <- Replace '1' with actual ID
)

### Step 4: Count and remove establishments in Dover

In [ ]:
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Establishments in Dover before deletion: {dover_count}")

establishments.delete_many({"LocalAuthorityName": "Dover"})

dover_count_after = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Establishments in Dover after deletion: {dover_count_after}")

### Step 5: Convert latitude and longitude to decimal numbers

In [ ]:
establishments.update_many(
    {},
    [{
        "$set": {
            "geocode.latitude": {"$toDouble": "$geocode.latitude"},
            "geocode.longitude": {"$toDouble": "$geocode.longitude"}
        }
    }]
)

### Step 6: Convert RatingValue to integers

In [ ]:
establishments.update_many(
    {},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)